<a href="https://colab.research.google.com/github/achraf-halla/ADME/blob/Solubility-Property/Copy_of_Solubility_AqSolDB_dc_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**We will continue to work on the task of predicting the solubilty of Molucules using the the "AqSolDB Solubility" dataset from TDC, but this Time we will try to leverage some helpful tools from deepchem library**

In [1]:
pip install PyTDC

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.2/154.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 522.9 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# **Data Loading**

In [3]:
from tdc.single_pred import ADME
import pandas as pd
dataset =  ADME(name="Solubility_AqSolDB")

Downloading...
100%|██████████| 853k/853k [00:00<00:00, 7.65MiB/s]
Loading...
Done!


In [4]:
pip install --pre deepchem[torch]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.1/823.1 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 37.2 MB/s eta 0:00:00


In [5]:
pip install  torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.8 MB/s eta 0:00:00


# **Experiment 1: using MultitaskFitTransformRegressor**

# Preprocessing

In [2]:
pip install deepchem

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.9 MB/s eta 0:00:00


In [ ]:
import deepchem as dc

In [ ]:
import numpy as np

with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:
    dataset.get_data().to_csv(tmpfile.name, index=False)
    featurizer = dc.feat.RDKitDescriptors()
    loader = dc.data.CSVLoader(["Y"], feature_field="Drug", featurizer=featurizer)
    data = loader.create_dataset(tmpfile.name)

X_clean = []
y_clean = []

for x, y in zip(data.X, data.y):
    if not np.any(np.isnan(x)):
        X_clean.append(x)
        y_clean.append(y)

from deepchem.data import NumpyDataset
clean_data = NumpyDataset(X=np.array(X_clean), y=np.array(y_clean))


[18:25:41] WARNING: not removing hydrogen atom without neighbors
[18:25:41] WARNING: not removing hydrogen atom without neighbors
[18:25:41] WARNING: not removing hydrogen atom without neighbors
[18:25:42] WARNING: not removing hydrogen atom without neighbors
[18:25:42] WARNING: not removing hydrogen atom without neighbors
[18:25:42] WARNING: not removing hydrogen atom without neighbors
[18:25:42] WARNING: not removing hydrogen atom without neighbors
[18:25:42] WARNING: not removing hydrogen atom without neighbors
[18:25:42] WARNING: not removing hydrogen atom without neighbors
[18:25:44] WARNING: not removing hydrogen atom without neighbors
[18:25:44] WARNING: not removing hydrogen atom without neighbors
[18:25:44] WARNING: not removing hydrogen atom without neighbors
[18:25:44] WARNING: not removing hydrogen atom without neighbors
[18:25:44] WARNING: not removing hydrogen atom without neighbors
[18:25:44] WARNING: not removing hydrogen atom without neighbors
[18:25:44] WARNING: not r

In [1]:
len(data), len(clean_data)

NameError: name 'data' is not defined

In [ ]:
from deepchem.splits import IndexSplitter
splitter = IndexSplitter()
train, valid, test = splitter.train_valid_test_split(clean_data)

In [ ]:
train

<NumpyDataset X.shape: (7278, 210), y.shape: (7278, 1), w.shape: (7278, 1), task_names: [0]>

In [ ]:
from torch.utils.data import DataLoader


train_loader = DataLoader(train, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid, batch_size=32)
test_loader = DataLoader(test, batch_size=32)

# Training

In [ ]:
assert not np.any(np.isnan(train.X)), "NaNs still present in features"

In [ ]:
from sklearn.metrics import mean_absolute_error
params_dict = {
    'n_tasks': [1],
    'n_features': [210],
    'layer_sizes': [[50], [100], [100, 100]],
    'dropouts': [0.2, 0.5],
    'learning_rate': [0.001, 0.0001],
    'batch_size' : [32]
}
optimizer = dc.hyper.GridHyperparamOpt(dc.models.MultitaskFitTransformRegressor)
metric = dc.metrics.Metric(mean_absolute_error)
best_model, best_hyperparams, all_results = optimizer.hyperparam_search(
        params_dict, train, valid, metric)

In [ ]:
all_results

{'_batch_size_32_dropouts_0.200000_layer_sizes[50]_learning_rate_0.001000_n_features_210_n_tasks_1': 0.712506358219896,
 '_batch_size_32_dropouts_0.200000_layer_sizes[50]_learning_rate_0.000100_n_features_210_n_tasks_1': 0.808909532337431,
 '_batch_size_32_dropouts_0.500000_layer_sizes[50]_learning_rate_0.001000_n_features_210_n_tasks_1': 0.8319260691883801,
 '_batch_size_32_dropouts_0.500000_layer_sizes[50]_learning_rate_0.000100_n_features_210_n_tasks_1': 0.8614483289164502,
 '_batch_size_32_dropouts_0.200000_layer_sizes[100]_learning_rate_0.001000_n_features_210_n_tasks_1': 0.7493239406651193,
 '_batch_size_32_dropouts_0.200000_layer_sizes[100]_learning_rate_0.000100_n_features_210_n_tasks_1': 0.7661299764330571,
 '_batch_size_32_dropouts_0.500000_layer_sizes[100]_learning_rate_0.001000_n_features_210_n_tasks_1': 0.7684160716130184,
 '_batch_size_32_dropouts_0.500000_layer_sizes[100]_learning_rate_0.000100_n_features_210_n_tasks_1': 0.8628482274136701,
 '_batch_size_32_dropouts_0.20

In [ ]:
from deepchem.models import TorchModel
from deepchem.models.losses import L1Loss
from deepchem.metrics import Metric, mae_score
from deepchem.models.callbacks import ValidationCallback
model = dc.models.MultitaskFitTransformRegressor(n_tasks=1,
                                      n_features=210,
                                      layer_sizes=[1000],
                                      dropouts=0.2,

                                      learning_rate=0.0001)
metric = Metric(mae_score, mode="regression")

val_cb = ValidationCallback(
    valid,
    interval=1000,
    metrics=[metric]
)

# Fit model with proper callbacks list
model.fit(train, nb_epoch=32, callbacks=[val_cb])




Step 1000 validation: mae_score=1.4144
Step 2000 validation: mae_score=0.821678
Step 3000 validation: mae_score=0.722966
Step 4000 validation: mae_score=0.700989


22.627365112304688

In [ ]:
classification_metric = Metric(mae_score,task_averager=np.mean)
model.evaluate(test, [classification_metric])

{'mean-mae_score': 0.8903336342465337}